In [1]:
import pandas as pd
import numpy as np
import yfinance as yf


In [2]:
# Download BTC data
btc = yf.download(
    'BTC-USD',
    start='2022-01-01',
    end='2025-01-01',
    interval='1d',
    auto_adjust=False
)
btc.columns = btc.columns.get_level_values(0)


[*********************100%***********************]  1 of 1 completed


## Indicator Functions

In [3]:
def ROC(price, timeperiod):
    return ((price - price.shift(timeperiod)) / price.shift(timeperiod)) * 100


In [4]:
def NVI(close, volume):
    nvi = [1000]
    for i in range(1, len(close)):
        if volume.iloc[i] < volume.iloc[i - 1]:
            change = (close.iloc[i] - close.iloc[i - 1]) / close.iloc[i - 1]
            nvi.append(nvi[-1] * (1 + change))
        else:
            nvi.append(nvi[-1])
    return pd.Series(nvi, index=close.index)


In [5]:
def PSAR(high, low, step=0.02, max_step=0.2):
    sar = [low.iloc[0]]
    ep = high.iloc[0]
    af = step
    uptrend = True
    for i in range(1, len(high)):
        prev_sar = sar[-1]
        if uptrend:
            new_sar = prev_sar + af * (ep - prev_sar)
            if low.iloc[i] < new_sar:
                uptrend = False
                new_sar = ep
                ep = low.iloc[i]
                af = step
            else:
                if high.iloc[i] > ep:
                    ep = high.iloc[i]
                    af = min(af + step, max_step)
        else:
            new_sar = prev_sar + af * (ep - prev_sar)
            if high.iloc[i] > new_sar:
                uptrend = True
                new_sar = ep
                ep = high.iloc[i]
                af = step
            else:
                if low.iloc[i] < ep:
                    ep = low.iloc[i]
                    af = min(af + step, max_step)
        sar.append(new_sar)
    return pd.Series(sar, index=high.index)


In [6]:
btc['ROC'] = ROC(btc['Close'], timeperiod=14)
btc['NVI'] = NVI(btc['Close'], btc['Volume'])
btc['PSAR'] = PSAR(btc['High'], btc['Low'])


START OF Q2

## Strategy A (STRICT)

In [7]:
btc.loc[
    (btc["ROC"] > 0) &
    (btc["NVI"] > btc["NVI"].shift(1)) &
    (btc["Close"] > btc["PSAR"]),
    "Signal"
] = 1

In [8]:
btc.loc[
    (btc["Close"] < btc["PSAR"]) |
    (btc["ROC"] < 0),
    "Signal"
] = -1

In [9]:
btc["Position"] = 0
btc["Entry_Price"] = 0.0
btc["Exit_Price"] = 0.0
btc["Returns"] = 0.0

In [10]:
stop_loss_pct = 0.02

In [11]:
in_trade = False
entry_price = 0

for i in range(1, len(btc)):

    # ENTRY
    if not in_trade and btc["Signal"].iloc[i] == 1:
        in_trade = True
        entry_price = btc["Close"].iloc[i]
        btc.at[btc.index[i], "Entry_Price"] = entry_price

    # EXIT
    elif in_trade:
        current_price = btc["Close"].iloc[i]

        # Stop-loss
        if current_price <= entry_price * (1 - stop_loss_pct):
            btc.at[btc.index[i], "Exit_Price"] = current_price
            btc.at[btc.index[i], "Returns"] = (current_price - entry_price) / entry_price
            in_trade = False
            entry_price = 0

        # Indicator-based exit
        elif btc["Signal"].iloc[i] == -1:
            btc.at[btc.index[i], "Exit_Price"] = current_price
            btc.at[btc.index[i], "Returns"] = (current_price - entry_price) / entry_price
            in_trade = False
            entry_price = 0


## Strategy D (RELAXED 2/3)

In [13]:
agreement = (
    (btc['ROC'] > 0).astype(int) +
    (btc['NVI'] > btc['NVI'].shift(1)).astype(int) +
    (btc['Close'] > btc['PSAR']).astype(int)
)
btc['Signal_D'] = -1
btc.loc[agreement >= 2, 'Signal_D'] = 1


## Backtest Functions

In [14]:
def run_strategy_D(btc, stop_loss_pct):
    df = btc.copy()
    df['Returns'] = 0.0
    in_trade = False
    entry_price = 0
    for i in range(1, len(df)):
        if not in_trade and df['Signal_D'].iloc[i] == 1:
            in_trade = True
            entry_price = df['Close'].iloc[i]
        elif in_trade:
            price = df['Close'].iloc[i]
            if price <= entry_price * (1 - stop_loss_pct) or df['Signal_D'].iloc[i] == -1:
                df.at[df.index[i], 'Returns'] = (price - entry_price) / entry_price
                in_trade = False
                entry_price = 0
    trades = df['Returns'][df['Returns'] != 0]
    sharpe = trades.mean() / trades.std()
    sortino = trades.mean() / trades[trades < 0].std()
    equity = (1 + df['Returns']).cumprod()
    max_dd = (equity / equity.cummax() - 1).min()
    return {
        'Trades': len(trades),
        'Net Profit': trades.sum(),
        'Sharpe': sharpe,
        'Sortino': sortino,
        'Max Drawdown': max_dd
    }


## Results

In [15]:
total_trades = (btc["Returns"] != 0).sum()
average_return = btc["Returns"][btc["Returns"] != 0].mean()
cumulative_return = (1 + btc["Returns"]).prod() - 1

In [16]:
# ----- STRATEGY A RESULTS -----

trade_returns = btc["Returns"][btc["Returns"] != 0]

num_winning_trades = (trade_returns > 0).sum()
num_losing_trades = (trade_returns < 0).sum()
total_profit = trade_returns[trade_returns > 0].sum()
total_loss = trade_returns[trade_returns < 0].sum()
net_profit = trade_returns.sum()
sharpe_ratio = trade_returns.mean() / trade_returns.std()
downside_returns = trade_returns[trade_returns < 0]
sortino_ratio = trade_returns.mean() / downside_returns.std()
btc["Equity"] = (1 + btc["Returns"]).cumprod()
rolling_max = btc["Equity"].cummax()
drawdown = (btc["Equity"] - rolling_max) / rolling_max
max_drawdown = drawdown.min()
print("Total Trades:", len(trade_returns))
print("Winning Trades:", num_winning_trades)
print("Losing Trades:", num_losing_trades)
print("Total Profit:", round(total_profit, 4))
print("Net Profit:", round(net_profit, 4))
print("Sharpe Ratio:", round(sharpe_ratio, 4))
print("Sortino Ratio:", round(sortino_ratio, 4))
print("Maximum Drawdown:", round(max_drawdown, 4))

Total Trades: 41
Winning Trades: 17
Losing Trades: 24
Total Profit: 1.2661
Net Profit: 0.498
Sharpe Ratio: 0.1603
Sortino Ratio: 0.6599
Maximum Drawdown: -0.2695


In [17]:
results = []
for sl in np.arange(0.001, 0.101, 0.001):
    r = run_strategy_D(btc, sl)
    r['Stop Loss'] = sl
    results.append(r)
opt_df = pd.DataFrame(results)
opt_df.sort_values('Net Profit', ascending=False).head()


,Trades,Net Profit,Sharpe,Sortino,Max Drawdown,Stop Loss
17,106,0.885894,0.119933,0.417158,-0.288803,0.018
21,104,0.884896,0.119740,0.403396,-0.302497,0.022
93,98,0.875720,0.122890,0.395083,-0.319163,0.094
92,98,0.875720,0.122890,0.395083,-0.319163,0.093
68,98,0.875720,0.122890,0.395083,-0.319163,0.069


In [21]:
clean_result_D = {
    "Trades": int(result_D["Trades"]),
    "Net Profit": float(result_D["Net Profit"]),
    "Sharpe": float(result_D["Sharpe"]),
    "Sortino": float(result_D["Sortino"]),
    "Max Drawdown": float(result_D["Max Drawdown"])
}

clean_result_D

{'Trades': 106,
 'Net Profit': 0.8858938460423972,
 'Sharpe': 0.11993276146920755,
 'Sortino': 0.4171579446921323,
 'Max Drawdown': -0.28880330273658616}

In Q2, I implemented 2 strategies. In the first strategy, the buying condition was buy when all 3 indicators were in favour and sell when anyone unfavours. For risk management, sell as soon as loss is more than 2%.
For strategy 2, I relaxed my buying and selling conditions a bit. I bought when any 2/3 indicators favoured and sold when atleast 2 unfavoured. To lower the risk, I tightened the risk managment to 1.8%.